In [1]:
import pandas as pd
from preprocessing.features.feat_cat import *
import numpy as np
import os
from load_data import *

In [127]:
dateparse_ep =  lambda x: datetime.strptime(x, '%Y%m%d%H')
def dateparse_fm(d):
    return pd.to_datetime(d)
#dateparse_fm = lambda x: datetime.strptime(x, '%d.%m.%Y %H:%M')

In [132]:
# Read EP data
EP17 = pd.read_csv(os.path.join(os.getcwd(), 'data_new/data', 'EP17.csv'),
                        encoding= 'unicode_escape', low_memory=False, parse_dates=['ein_dat_hr'], date_parser=dateparse_ep)

EP18 = pd.read_csv(os.path.join(os.getcwd(), 'data_new/data', 'EP18.csv'),
                        encoding= 'unicode_escape', low_memory=False, parse_dates=['ein_dat_hr'], date_parser=dateparse_ep)

EP19 = pd.read_csv(os.path.join(os.getcwd(), 'data_new/data', 'EP19.csv'),
                        encoding= 'unicode_escape', low_memory=False, parse_dates=['ein_dat_hr'], date_parser=dateparse_ep)

In [128]:
# Read FM data
FM17 = pd.read_csv(os.path.join(os.getcwd(), 'data_new/data', 'FM17.csv'),
                        low_memory=False, parse_dates=['datum_beg_fm_zeit_beg_fm'], date_parser=dateparse_fm)
FM18 = pd.read_csv(os.path.join(os.getcwd(), 'data_new/data', 'FM18.csv'),
                        low_memory=False, parse_dates=['datum_beg_fm_zeit_beg_fm'], date_parser=dateparse_fm)
FM19 = pd.read_csv(os.path.join(os.getcwd(), 'data_new/data', 'FM19.csv'), 
                        low_memory=False, parse_dates=['datum_beg_fm_zeit_beg_fm'], date_parser=dateparse_fm)

In [133]:
# Focus on adult patients < 65
EP17 = EP17[(EP17['alter'] >= 18) & (EP17['alter'] < 65)]
EP18 = EP18[(EP18['alter'] >= 18) & (EP18['alter'] < 65)]
EP19 = EP19[(EP19['alter'] >= 18) & (EP19['alter'] < 65)]

In [5]:
def store_back(df, name):
    df.to_csv('data_new/' + name + '.csv', index=False)

#grouped = EP17.groupby('bur_knr')['fm_betroffen'].sum()
#ind = list(grouped.index[grouped.eq(0)])
#EP17 = EP17[~EP17['bur_knr'].isin(ind)] #9857

#grouped = EP18.groupby('bur_knr')['fm_betroffen'].sum()
#ind = list(grouped.index[grouped.eq(0)])
#EP18 = EP18[~EP18['bur_knr'].isin(ind)] #12425

#grouped = EP19.groupby('anq_id')['fm_betroffen'].sum()
#ind = list(grouped.index[grouped.eq(0)])
#EP19 = EP19[~EP19['anq_id'].isin(ind)] #11258

In [134]:
# Preselect EP features, focus on admission data 
#to_drop = zm + other + med_out + klin_out + bhdlg_out + beh + beh_out + bscl_diff + h_diff  + b_out + bscl_out + h_out + h_ges_ein + dropout_out + datum + datum_out + person_out + dummy + dummy_out + qual + qual_out + diagnose_out

to_drop = zm + other + med + med_out + klin_out + bhdlg + bhdlg_out + beh + beh_out + bscl_diff + h_diff + b_ein + b_out + bscl_out + h_out + bscl_ein + h_ges_ein + dropout_out + datum + datum_out + person_out + dummy + dummy_out + qual + qual_out + diagnose_out
EP17 = EP17.drop(columns = to_drop + ['klinik'])
EP18 = EP18.drop(columns = to_drop)
EP19 = EP19.drop(columns = to_drop)
EP18 = EP18.drop(columns = [item for item in EP18.columns if item not in EP17.columns])
#EP19 = EP19.drop(columns = [item for item in EP19.columns if item not in EP17.columns])

In [135]:
#Preselect FM features, use 'difftime_fm_start_ein_dat' instead of manually computing delay from date features # ','datum_beg_fm_zeit_beg_fm'' kept
FM17 = FM17.drop(columns=['fm','ein_dat_d','bur_fm','datenlieferung_fm','Filter_1','datum_ende_fm_jahr','datum_beg_fm_jahr','zeit_beg_fm','check_fid','dat_beg_fehlt','zeit_beg_fehlt','datum_ende_fehlt', 'zeit_ende_fehlt','aus_dat_d', 'aus_dat_d_year', 'ein_dat_d_year', 'subset_var_1', 'subset_var_2', 'fm_qual', 'erhebung_1','standort','datum_ende_fm_zeit_ende_fm','datum_ende_fm','zeit_ende_fm'])
FM18 = FM18.drop(columns=['fm','ein_dat_d','bur_fm','datenlieferung_fm','check_fid','Filter_1','datum_ende_fm_jahr','datum_beg_fm_jahr','zeit_beg_fm','dat_beg_fehlt','zeit_beg_fehlt','datum_ende_fehlt', 'zeit_ende_fehlt','aus_dat_d', 'aus_dat_d_year', 'ein_dat_d_year', 'subset_var_1', 'subset_var_2', 'fm_qual', 'erhebung_1','standort','datum_ende_fm_zeit_ende_fm','datum_ende_fm','zeit_ende_fm'])
FM19 = FM19.drop(columns=['fm','ein_dat_d','bur_fm','datenlieferung_fm','datum_ende_fm_jahr','datum_beg_fm_jahr','check_fid','zeit_beg_fm','dat_beg_fehlt','zeit_beg_fehlt','datum_ende_fehlt', 'zeit_ende_fehlt','aus_dat_d', 'aus_dat_d_year', 'ein_dat_d_year', 'subset_var_1', 'subset_var_2', 'fm_qual', 'erhebung_1','standort','datum_ende_fm_zeit_ende_fm','datum_ende_fm','zeit_ende_fm'])

#For simplicity, drop the handful of missing data that remains 
FM17 = FM17.dropna()
FM18 = FM18.dropna()
FM19 = FM19.dropna()

FM17 = FM17.rename(columns={'difftime_fm_start_ein_dat': 'delay'})
FM18 = FM18.rename(columns={'difftime_fm_start_ein_dat': 'delay'})
FM19 = FM19.rename(columns={'difftime_fm_start_ein_dat': 'delay'})

#compute delay between admission and coercive measure in days
FM17.delay = FM17.delay/-3600 #-86400.0
FM18.delay = FM18.delay/-3600 #-86400.0
FM19.delay = FM19.delay/-3600 #-86400.0


In [136]:
# A few patients had wrong dates entered as start of coercive measure, chose to set them to 0
FM17.delay = FM17.delay.apply(lambda x: 0 if x < 0 else x)
FM18.delay = FM18.delay.apply(lambda x: 0 if x < 0 else x)
FM19.delay = FM19.delay.apply(lambda x: 0 if x < 0 else x)

FM17['year'] = 2017
FM18['year'] = 2018
FM19['year'] = 2019

In [137]:
EP17['year'] = 2017
EP18['year'] = 2018
EP19['year'] = 2019

# missed column elements to be dropped in EP data #'ein_dat_hr kept
col = ['stat_fall', 'psych_ds', 'pat_ds', 'kant_ds','ph_ein_missing', 'ph_aus_missing', 'pb_ein_missing','pb_aus_missing', 'daten_ohne_fid_subset','festhalten']
EP17 = EP17.drop(columns=col)
EP18 = EP18.drop(columns=col)
EP19 = EP19.drop(columns=col)

In [138]:
# Replace 99 in occupational data by nan
tmp1 = ['bildung','zivilstand','besch_teil','besch_voll','besch_nicht','besch_haus','besch_ausb','besch_reha','besch_iv_ahv','besch_gesch','besch_unbek','ffe']
EP17[tmp1] = EP17[tmp1].replace(99,np.nan)
EP18[tmp1] = EP18[tmp1].replace(99,np.nan)
EP19[tmp1] = EP19[tmp1].replace(99,np.nan)

# set "occupation unknown" to 1, if nan
EP17['besch_unbek'] = EP17['besch_unbek'].fillna(1)
EP18['besch_unbek'] = EP18['besch_unbek'].fillna(1)
EP19['besch_unbek'] = EP19['besch_unbek'].fillna(1)

# fill out missing data in number of coercive measures applied by 0
tmp2 = ['anzahl_fm_fall_original','isolation_anzahl_original','fixierung_anzahl_original','medi_gesamt_anzahl_original']
EP17[tmp2] = EP17[tmp2].fillna(0)
EP18[tmp2] = EP18[tmp2].fillna(0)
EP19[tmp2] = EP19[tmp2].fillna(0)
EP19['geschl'] = EP19['geschl'].fillna(1)
EP17['h8_art_ein_num'] = EP17['h8_art_ein_num'].fillna(99)
EP18['h8_art_ein_num'] = EP18['h8_art_ein_num'].fillna(99)
EP19['h8_art_ein_num'] = EP19['h8_art_ein_num'].fillna(99)

# replace missing HoNOS data by 9
tmp3 = ['dropout_ph_ein','h1_ein','h2_ein','h3_ein','h4_ein','h5_ein','h6_ein','h7_ein','h8_ein','h9_ein','h10_ein','h11_ein','h12_ein','dropout_pb_ein']
EP17[tmp3] = EP17[tmp3].fillna(9)
EP18[tmp3] = EP18[tmp3].fillna(9)
EP19[tmp3] = EP19[tmp3].fillna(9)

In [ ]:
# optional, to be ignored 
EP17.datum_ph_ein_d = pd.to_datetime(EP17.datum_ph_ein_d, infer_datetime_format=True)
EP18.datum_ph_ein_d = pd.to_datetime(EP18.datum_ph_ein_d, infer_datetime_format=True)
EP19.datum_ph_ein_d = pd.to_datetime(EP19.datum_ph_ein_d, infer_datetime_format=True)
EP17.ein_dat_d = pd.to_datetime(EP17.ein_dat_d, infer_datetime_format=True)
EP18.ein_dat_d = pd.to_datetime(EP18.ein_dat_d, infer_datetime_format=True)
EP19.ein_dat_d = pd.to_datetime(EP19.ein_dat_d, infer_datetime_format=True)
FM17.ein_dat_d = pd.to_datetime(FM17.ein_dat_d, infer_datetime_format=True)
FM18.ein_dat_d = pd.to_datetime(FM18.ein_dat_d, infer_datetime_format=True)
FM19.ein_dat_d = pd.to_datetime(FM19.ein_dat_d, infer_datetime_format=True)
EP17['datum_ph_ein_d'] = EP17['datum_ph_ein_d'].fillna(EP17['ein_dat_d'])
EP18['datum_ph_ein_d'] = EP18['datum_ph_ein_d'].fillna(EP18['ein_dat_d'])
EP19['datum_ph_ein_d'] = EP19['datum_ph_ein_d'].fillna(EP19['ein_dat_d'])
EP17 = EP17.dropna() #remove last nan

In [139]:
# a few features in EP19 are not present in EP17 and EP18
EP19 = EP19.drop(columns=['kliniktyp','klinik_typ','bb_unbek_fehlt','bb_FP','bb_abhPsyy','bb_kjp','bb_allgPsy','bb_altersPsy','bur_knr','bur_knr_2c','kurzaufenth_ph','kurzaufenth_pb'])

In [140]:
# choose 9 to fill out missing data, 9 is already "unknown" category in "Bildung" and "Zivilstand"
EP17.zivilstand = EP17.zivilstand.fillna(9)
EP17.bildung = EP17.bildung.fillna(9)
EP17.ffe = EP17.ffe.fillna(9)
EP18.zivilstand = EP18.zivilstand.fillna(9)
EP18.bildung = EP18.bildung.fillna(9)
EP18.ffe = EP18.ffe.fillna(9)
EP19.zivilstand = EP19.zivilstand.fillna(9)
EP19.bildung = EP19.bildung.fillna(9)
EP19.ffe = EP19.ffe.fillna(9)

In [141]:
EP19 = EP19[EP19['anq_id'] < 100]

In [40]:
# Build our full EP dataset by concatenating EP17, EP18 and EP19
EP_prep = pd.concat([EP17, EP18, EP19])
EP_prep = EP_prep.reset_index()
EP_prep = EP_prep.drop(columns=['index','bur_knr','anq_id'])

# Remove clinics that do not apply coercive measures from the resulting EP dataset
grouped = EP_prep.groupby('klinik_anonym')['fm_betroffen'].sum()
ind = list(grouped.index[grouped.eq(0)])
EP_prep_final = EP_prep[~EP_prep['klinik_anonym'].isin(ind)]

# drop some features that we forgot to eliminate
EP_prep_final = EP_prep_final.drop(columns=['datum_ph_ein_d','behandlungsbereich','ein_dat_d'])

# Replace the last missing data (occupational data) by 0 (false)
EP_prep_final = EP_prep_final.fillna(0)

In [14]:
store_back(EP_prep_final,'EP_d')

_____________________________________________________________________________________________________________

In [142]:
# Merge our EP and FM datasets, by doing a left join
EM17 = pd.merge(EP17, FM17, left_on = ['year','bur_knr','FID'],
        right_on=['year','bur_knr_fm','FID'], how='left')
EM18 = pd.merge(EP18, FM18, left_on = ['year','bur_knr','FID'],
        right_on=['year','bur_knr_fm','FID'], how='left')
EM19 = pd.merge(EP19, FM19, left_on = ['year','anq_id','FID'],
        right_on=['year','anq_id','FID'], how='left')

In [143]:
# if no coercive measure applied, set "art_fm" to 0
EM17.art_fm = EM17.art_fm.fillna(0)
EM18.art_fm = EM18.art_fm.fillna(0)
EM19.art_fm = EM19.art_fm.fillna(0)

# if no coercive measure applied, set "delay" to -1
EM17.delay = EM17.delay.fillna(-1)
EM18.delay = EM18.delay.fillna(-1)
EM19.delay = EM19.delay.fillna(-1)

# drop merging features we won't use anymore
EM17 = EM17.drop(columns = ['bur_knr_fm','bur_knr'])
EM18 = EM18.drop(columns = ['bur_knr_fm','bur_knr'])
EM19 = EM19.drop(columns = ['anq_id'])

# Concatenate the merged datasets from the 3 years
EM_prep = pd.concat([EM17, EM18, EM19])
EM_prep = EM_prep.reset_index()
EM_prep = EM_prep.drop(columns=['index'])

# As above, remove clinics that do not perform coercive measures
grouped = EM_prep.groupby('klinik_anonym')['fm_betroffen'].sum()
ind = list(grouped.index[grouped.eq(0)])
EM_prep = EM_prep[~EM_prep['klinik_anonym'].isin(ind)] 

In [114]:
## add process_time_delay
load = Preprocessor()
mask = (EM_prep.delay == 0)
df_0 = EM_prep[mask]

#EM_prep.datum_beg_fm_zeit_beg_fm = pd.to_datetime(EM_prep.ein_dat_d, infer_datetime_format=True)
#EM_prep.ein_dat_hr= pd.to_datetime(EM_prep.ein_dat_d, infer_datetime_format=True)

#EM_prep['period'] = EM_prep['delay']
EM_prep['period'] = EM_prep['datum_beg_fm_zeit_beg_fm'].subtract(EM_prep['ein_dat_hr'])

In [149]:
x = EM_prep['datum_beg_fm_zeit_beg_fm'].subtract(EM_prep['ein_dat_hr']) / pd.Timedelta('1 hour')

In [152]:
EM_prep['period'] = x

In [158]:
EM_prep.period = EM_prep.period.fillna(-1)

In [155]:
(EM_prep.period >= 0).sum()

40897

In [159]:
EM_min = EM_prep.loc[EM_prep.groupby(['year','klinik_anonym','FID'])['period'].idxmin()]


In [162]:
EM_max = EM_prep.loc[EM_prep.groupby(['year','klinik_anonym','FID'])['delay'].idxmax()]

In [168]:
((EM_min.period >= 0) & (EM_min.period < 24)).sum()

4848

In [169]:
((EM_min.delay >= 0) & (EM_min.delay < 24)).sum()

5818

In [171]:
(EM_min.delay >= 24).sum()

6528

In [160]:
EM_min.fm_betroffen.sum()

12434

In [165]:
(EM_max.period >= 24).sum()

7270

In [161]:
(EM_min.period >= 24).sum()

5119

In [96]:
tmp = EM_min[(EM_min.delay >= 0) & (EM_min.delay < 1)]
EM_min2 = EM_min.drop(tmp.index) # drop 6353

_____________________________________________________________________________________________

In [1114]:
# Build dataset for target: *first* coercive measure after at least 24h 
EM_min = EM_prep.loc[EM_prep.groupby(['year','klinik_anonym','FID'])['delay'].idxmin()]

# Remove cases with *first* coercive measure before 24h
tmp = EM_min[(EM_min.delay >= 0) & (EM_min.delay < 1)]
EM_min = EM_min.drop(tmp.index) # drop 6353

EM_min = EM_min.drop(columns=['datum_ph_ein_d','behandlungsbereich','ein_dat_d_x','ein_dat_d_y','datum_beg_fm'])
EM_min = EM_min.fillna(0)

In [1116]:
store_back(EM_min,'EM_min_d')

________________________

In [1125]:
# Build dataset for target: Case with coercive measure after at least 24h 
EM_max = EM_prep.loc[EM_prep.groupby(['year','klinik_anonym','FID'])['delay'].idxmax()]

# Remove cases with coercive measures before 24h
EM_max = EM_max[(EM_max.delay >= 1) | (EM_max.delay == -1)] # 4178

EM_max = EM_max.drop(columns=['datum_ph_ein_d','behandlungsbereich','ein_dat_d_x','ein_dat_d_y','datum_beg_fm'])
EM_max = EM_max.fillna(0)


In [1130]:
store_back(EM_max, 'EM_max_d')

___________________________________________________

In [1045]:
# Optionally, we may merge all occupational dummy variables into one categorical variable - to ignore for now
tmp = ['besch_teil','besch_voll','besch_nicht','besch_haus','besch_ausb','besch_reha','besch_iv_ahv','besch_gesch','besch_unbek']
#EP_prep_final[tmp] = EPmin[tmp].replace(99,9)
EM_max = EM_max.rename(columns={'besch_voll':1,'besch_teil':2,'besch_nicht':3,'besch_haus':4,'besch_ausb':5,'besch_reha':6,'besch_iv_ahv':7,'besch_gesch':8,'besch_unbek':9})

tmp2 = [1,2,3,4,5,6,7,8,9]
x = EM_max[tmp2].idxmax(axis=1)

EM_max = EM_max.drop(columns=tmp2)
EM_max['besch'] = x

#EPmin = EPmin.drop(columns=['ein_dat_d_x','datum_ph_ein_d'])

_________________________________________

Perform the last data corrections 

In [15]:
EP = pd.read_csv('data_new/EP_d.csv', low_memory=False)
#Emax = pd.read_csv('data_new/EM_max_d.csv', low_memory=False)
#Emin = pd.read_csv('data_new/EM_min_d.csv', low_memory=False)

In [16]:
EP.columns

Index(['FID', 'klinik_anonym', 'dropout_ph_ein', 'h1_ein', 'h2_ein', 'h3_ein',
       'h4_ein', 'h5_ein', 'h6_ein', 'h7_ein',
       ...
       'bscl_zwa_ein', 'bscl_soz_ein', 'bscl_dep_ein', 'bscl_ang_ein',
       'bscl_agg_ein', 'bscl_pho_ein', 'bscl_par_ein', 'bscl_psy_ein',
       'anz_nebendiagnosen', 'year'],
      dtype='object', length=118)

In [17]:
# Dropout pb, patients too young => andere (9 cases)
EP.dropout_pb_ein = np.where(EP.dropout_pb_ein == 5.0, 8.0, EP.dropout_pb_ein)

EP.ort_v_ein = np.where(EP.ort_v_ein == 99, 9, EP.ort_v_ein)

EP.ein_art = np.where(EP.ein_art == 99, 9, EP.ein_art)

# strange categories that do not appear on the codebook => unknown (25 cases)
EP.einw_inst = np.where(EP.einw_inst.isin([99,84,7]), 9, EP.einw_inst)

# 99 => nicht näher bezeichnete psy. störung (643 cases)
EP.hpt_diagn_num = np.where(EP.hpt_diagn_num == 99, 10, EP.hpt_diagn_num)

In [18]:
store_back(EP, 'EP_d')

_________________________________________________________________________________________________________________________________________________

Perform feature selection on HoNOS based on our logistic regression results

In [127]:
EP = pd.read_csv('data_new/EP_d.csv', low_memory=False)
#Emax = pd.read_csv('data_new/EM_max_d.csv', low_memory=False)
#Emin = pd.read_csv('data_new/EM_min_d.csv', low_memory=False)

In [158]:
EP_prep = EP.drop(columns=['bildung','h9_ein','h11_ein','h12_ein','zivilstand'])

EP_prep = EP_prep[~EP_prep.h1_ein.eq(9)]
EP_prep = EP_prep[~EP_prep.h2_ein.eq(9)]
EP_prep = EP_prep[~EP_prep.h3_ein.eq(9)]
EP_prep = EP_prep[~EP_prep.h4_ein.eq(9)]
EP_prep = EP_prep[~EP_prep.h5_ein.eq(9)]
EP_prep = EP_prep[~EP_prep.h6_ein.eq(9)]
EP_prep = EP_prep[~EP_prep.h7_ein.eq(9)]
EP_prep = EP_prep[~EP_prep.h7_ein.eq(9)]
EP_prep = EP_prep[~EP_prep.h10_ein.eq(9)]

In [161]:
EP_prep.columns

Index(['FID', 'klinik_anonym', 'dropout_ph_ein', 'h1_ein', 'h2_ein', 'h3_ein',
       'h4_ein', 'h5_ein', 'h6_ein', 'h7_ein', 'h8_ein', 'h10_ein',
       'dropout_pb_ein', 'geschl', 'alter', 'ort_v_ein', 'ein_art',
       'einw_inst', 'PID', 'besch_teil', 'besch_voll', 'besch_nicht',
       'besch_haus', 'besch_ausb', 'besch_reha', 'besch_iv_ahv', 'besch_gesch',
       'besch_unbek', 'ffe', 'hpt_diagn_num', 'h8_art_ein_num', 'fm_betroffen',
       'anzahl_fm_fall_original', 'isolation_anzahl_original',
       'fixierung_anzahl_original', 'medi_gesamt_anzahl_original',
       'anz_nebendiagnosen', 'year'],
      dtype='object')

In [160]:
store_back(EP_prep,'EP_prep_d')

In [34]:
EP_prep.fm_betroffen.value_counts()

0    135260
1     10376
Name: fm_betroffen, dtype: int64